<div class="title">Meanshift and Camshift </div>  </div>
</div><!--header-->
<div class="contents">
<div class="textblock"><h2>Goal </h2>
<ul>
<li>We will learn about Meanshift and Camshift algorithms to find and track objects in videos.</li>
</ul>
<h2>Meanshift </h2>
<p>The intuition behind the meanshift is simple. Consider you have a set of points. (It can be a pixel distribution like histogram backprojection). You are given a small window ( may be a circle) and you have to move that window to the area of maximum pixel density (or maximum number of points). It is illustrated in the simple image given below:</p>
<div class="image">
<img src="https://docs.opencv.org/4.0.0/meanshift_basics.jpg" alt="meanshift_basics.jpg"/>
<div class="caption">
image</div></div>
<p> The initial window is shown in blue circle with the name "C1". Its original center is marked in blue rectangle, named "C1_o". But if you find the centroid of the points inside that window, you will get the point "C1_r" (marked in small blue circle) which is the real centroid of window. Surely they don't match. So move your window such that circle of the new window matches with previous centroid. Again find the new centroid. Most probably, it won't match. So move it again, and continue the iterations such that center of window and its centroid falls on the same location (or with a small desired error). So finally what you obtain is a window with maximum pixel distribution. It is marked with green circle, named "C2". As you can see in image, it has maximum number of points. The whole process is demonstrated on a static image below:</p>
<div class="image">
<img src="https://docs.opencv.org/4.0.0/meanshift_face.gif" alt="meanshift_face.gif"/>
<div class="caption">
image</div></div>
<p> So we normally pass the histogram backprojected image and initial target location. When the object moves, obviously the movement is reflected in histogram backprojected image. As a result, meanshift algorithm moves our window to the new location with maximum density.</p>
<h3>Meanshift in OpenCV.js</h3>
<p>To use meanshift in OpenCV.js, first we need to setup the target, find its histogram so that we can backproject the target on each frame for calculation of meanshift. We also need to provide initial location of window. For histogram, only Hue is considered here. Also, to avoid false values due to low light, low light values are discarded using <b><a class="el" href="../../d2/de8/group__core__array.html#ga48af0ab51e36436c5d04340e036ce981" title="Checks if array elements lie between the elements of two other arrays. ">cv.inRange()</a></b> function.</p>
<p>We use the function: <b><a class="el" href="../../dc/d6b/group__video__track.html#ga432a563c94eaf179533ff1e83dbb65ea" title="Finds an object on a back projection image. ">cv.meanShift</a> (probImage, window, criteria)</b> </p><dl class="params"><dt>Parameters</dt><dd>
  <table class="params">
    <tr><td class="paramname">probImage</td><td>Back projection of the object histogram. See <a class="el" href="../../d6/dc7/group__imgproc__hist.html#ga3a0af640716b456c3d14af8aee12e3ca" title="Calculates the back projection of a histogram. ">cv.calcBackProject</a> for details. </td></tr>
    <tr><td class="paramname">window</td><td>Initial search window. </td></tr>
    <tr><td class="paramname">criteria</td><td>Stop criteria for the iterative search algorithm. </td></tr>
  </table>
  </dd>
</dl>
<dl class="section return"><dt>Returns</dt><dd>number of iterations meanShift took to converge and the new location</dd></dl>
    


In [1]:
import numpy as np
import cv2 as cv
cap = cv.VideoCapture('C:/Users/piyus/All programming files/Computer_Vision/data/slow_traffic_small.mp4')
# take first frame of the video
ret, frame = cap.read()

# setup initial location of window
x, y, width, height = 300, 200, 100, 50
track_window = (x, y ,width, height)

# set up the ROI for tracking
roi = frame[y:y+height, x : x+width]
hsv_roi = cv.cvtColor(roi, cv.COLOR_BGR2HSV)
mask = cv.inRange(hsv_roi, np.array((0., 60., 32.)), np.array((180., 255., 255)))
roi_hist = cv.calcHist([hsv_roi], [0], mask, [180], [0, 180])
cv.normalize(roi_hist, roi_hist, 0, 255,cv.NORM_MINMAX)

# Setup the termination criteria, either 10 iteration or move by atleast 1 pt
term_crit = ( cv.TERM_CRITERIA_EPS | cv.TERM_CRITERIA_COUNT, 10, 1)
cv.imshow('roi',roi)
while(1):
    ret, frame = cap.read()
    if ret == True:

        hsv = cv.cvtColor(frame, cv.COLOR_BGR2HSV)
        dst = cv.calcBackProject([hsv], [0], roi_hist, [0, 180], 1)
        # apply meanshift to get the new location
        ret, track_window = cv.meanShift(dst, track_window, term_crit)
        # Draw it on image
        x,y,w,h = track_window
        final_image = cv.rectangle(frame, (x,y), (x+w, y+h), 255, 3)

        cv.imshow('dst', dst)
        cv.imshow('final_image',final_image)
        k = cv.waitKey(30) & 0xff
        if k == 27:
            break
    else:
        break
        